# Plotting Redbox

A place to draw the root graph as it exists in code.

In [1]:
%load_ext autoreload

%aimport redbox
from redbox.app import Redbox
from pathlib import Path
from dotenv import load_dotenv

_ = load_dotenv(Path.cwd().parent / ".env")

app = Redbox()

INFO:elastic_transport.transport:HEAD http://localhost:9200/_alias/redbox-data-chunk-current [status:200 duration:0.037s]


In [5]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        app.graph.get_graph(xray=True).draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

Traceback (most recent call last):
  File "_pydevd_bundle\\pydevd_cython.pyx", line 1610, in _pydevd_bundle.pydevd_cython.handle_exception
  File "/Users/mark.dunne/src/idot/redbox/redbox-core/.venv/lib/python3.12/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2197, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, trace_suspend_type, from_this_thread, frames_tracker)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mark.dunne/src/idot/redbox/redbox-core/.venv/lib/python3.12/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2266, in _do_wait_suspend
    notify_event.wait(wait_timeout)
  File "/Users/mark.dunne/.pyenv/versions/3.12.4/lib/python3.12/threading.py", line 655, in wait
    signaled = self._cond.wait(timeout)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mark.dunne/.pyenv/versions/3.12.4/lib/python3.12/threading.py", line 359

AttributeError: 'NoneType' object has no attribute 'id'

In [4]:
app.graph.nodes

{'__start__': <langgraph.pregel.read.PregelNode at 0x13ee437a0>,
 'p_search': <langgraph.pregel.read.PregelNode at 0x13ee43d70>,
 'p_search_agentic': <langgraph.pregel.read.PregelNode at 0x13ee43ce0>,
 'p_chat': <langgraph.pregel.read.PregelNode at 0x13ee43c20>,
 'p_chat_with_documents': <langgraph.pregel.read.PregelNode at 0x13ee43b60>,
 'p_retrieve_metadata': <langgraph.pregel.read.PregelNode at 0x13ee43aa0>,
 'd_keyword_exists': <langgraph.pregel.read.PregelNode at 0x13ee436e0>,
 'd_docs_selected': <langgraph.pregel.read.PregelNode at 0x13ee43590>}

In [10]:
app.graph.get_graph()

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'pydantic.v1.main.LangGraphInput'>, metadata=None), 'p_search': Node(id='p_search', name='p_search', data=<langgraph.graph.state.CompiledStateGraph object at 0x146214860>, metadata=None), 'p_search_agentic': Node(id='p_search_agentic', name='p_search_agentic', data=<langgraph.graph.state.CompiledStateGraph object at 0x146215130>, metadata=None), 'p_chat': Node(id='p_chat', name='p_chat', data=<langgraph.graph.state.CompiledStateGraph object at 0x1450d5b50>, metadata=None), 'p_chat_with_documents': Node(id='p_chat_with_documents', name='p_chat_with_documents', data=<langgraph.graph.state.CompiledStateGraph object at 0x146f19670>, metadata=None), 'p_retrieve_metadata': Node(id='p_retrieve_metadata', name='p_retrieve_metadata', data=<langgraph.graph.state.CompiledStateGraph object at 0x146217290>, metadata=None), 'd_keyword_exists': Node(id='d_keyword_exists', name='d_keyword_exists', data=d_keyword_exists(func_a

In [11]:
app.graph.get_graph??

Signature:
app.graph.get_graph(
    config: Optional[langchain_core.runnables.config.RunnableConfig] = None,
    *,
    xray: Union[int, bool] = False,
) -> langchain_core.runnables.graph.Graph
Source:   
    def get_graph(
        self,
        config: Optional[RunnableConfig] = None,
        *,
        xray: Union[int, bool] = False,
    ) -> DrawableGraph:
        """Returns a drawable representation of the computation graph."""
        graph = DrawableGraph()
        start_nodes: dict[str, DrawableNode] = {
            START: graph.add_node(self.get_input_schema(config), START)
        }
        end_nodes: dict[str, DrawableNode] = {}
        if xray:
            subgraphs = dict(self.get_subgraphs())
        else:
            subgraphs = {}

        def add_edge(
            start: str, end: str, label: Optional[str] = None, conditional: bool = False
        ) -> None:
            if end == END and END not in end_nodes:
                end_nodes[END] = graph.add_node(self.get_outp